In [ ]:
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, log_loss, hinge_loss, matthews_corrcoef
from sklearn.externals import joblib

%matplotlib inline

In [ ]:
DATA_DIR = "./data"
MODELS_DIR = "./models"
# Set the default random seed
np.random.seed(42)

# Load the whole labelled dataset

In [ ]:
data = pd.read_csv(DATA_DIR + "/dataset.csv")

In [ ]:
data.head()

# Plot a sample of the loaded dataset

In [ ]:
def plotNum(ind):
    plt.imshow(np.reshape(np.array(data.iloc[ind,1:]), (28, 28)), cmap="gray")

In [ ]:
plt.figure()
for ii in range(1,17):
    plt.subplot(4,4,ii)
    plotNum(ii)

# Split the dataset into train/test set

In [ ]:
train, test = train_test_split(data, test_size=0.1, stratify=data['label'])

# Sort instances by index

In [ ]:
train = train.sort_index()
test = test.sort_index()

# Store the train/test set to disk

In [ ]:
train.to_csv(DATA_DIR + "/train.csv", index=False, header=True)
test.to_csv(DATA_DIR + "/test.csv", index=False, header=True)

# Store the train/test set index to disk

In [ ]:
train.index.to_series().to_csv(DATA_DIR + "/train.idx", index=False, header=False)
test.index.to_series().to_csv(DATA_DIR + "/test.idx", index=False, header=False)

# Model Training

In [ ]:
X = train.iloc[:, 1:].values
y = train['label'].values

## Split Train/Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## Create the RandomForest classifier

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, max_depth=X_train.shape[1])

## Specify hyperparameters

In [ ]:
parameters = {'n_estimators':(10, 50, 100, 250, 500)}

## Run grid search cross-validation

In [ ]:
clf = GridSearchCV(rfc, parameters, cv=5, scoring='f1_micro', return_train_score=False, verbose=2)

In [ ]:
clf.fit(X_train, y_train)

## Print out the results

In [ ]:
clf.cv_results_

# Model Evaluation

## Compute predictions on the held-out portion of the dataset

In [ ]:
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)

## Balanced Accuracy

In [ ]:
print("Balanced Accuracy: {:.3f}".format(balanced_accuracy_score(y_test, y_pred)))

## F1 score (micro-averaged)

In [ ]:
print("F1 score (micro-averaged): {:.3f}".format(f1_score(y_test, y_pred, average='micro')))

## F1 score (macro-averaged)

In [ ]:
print("F1 score (macro-averaged): {:.3f}".format(f1_score(y_test, y_pred, average='macro')))

## Log Loss

In [ ]:
print("Log Loss: {:.3f}".format(log_loss(y_test, y_pred_proba)))

## Hinge Loss

In [ ]:
print("Hinge Loss: {:.3f}".format(hinge_loss(y_test, y_pred_proba)))

## Matthews Correlation Coefficient

In [ ]:
print("Matthews Correlation Coefficient: {:.3f}".format(matthews_corrcoef(y_test, y_pred)))

## Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
_ = plot_confusion_matrix(y_test, y_pred, classes=range(10),
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
_ = plot_confusion_matrix(y_test, y_pred, classes=range(10), normalize=True,
                      title='Normalized confusion matrix')


# Re-fit the best model to the whole training set

In [ ]:
clf.best_estimator_.fit(X, y)

# Save the best model

In [ ]:
with gzip.GzipFile(MODELS_DIR + "/RF_best.model.gz", "wb") as outfile:
    joblib.dump(clf.best_estimator_, outfile)